In [1]:
 !pip install seaborn
!pip install missingno
!pip install xgboost
!pip install catboost
!pip install regex
!pip install sklearn
!pip install pandas
!pip install numpy
!pip install imblearn
!pip install lightgbm

# Import packages 

In [48]:
import pandas as pd   # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np   # linear algebra
import matplotlib.pyplot as plt  #graphs and plots
import seaborn as sns   #data visualizations 
import csv # Some extra functionalities for csv  files - reading it as a dictionary
from lightgbm import LGBMClassifier #sklearn is for machine learning and statistical modeling including classification, regression, clustering and dimensionality reduction 

from sklearn.model_selection import train_test_split, cross_validate   #break up dataset into train and test sets

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler


# importing python library for working with missing data
import missingno as msno
# To install missingno use: !pip install missingno
import re    # This library is used to perform regex pattern matching

# import various functions from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, classification_report, make_scorer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
print("Successful")

Successful


In [49]:
from sklearn.model_selection import KFold,cross_val_score, RepeatedStratifiedKFold,StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import OneHotEncoder,StandardScaler,PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.compose import make_column_transformer
from imblearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score,\
                            precision_score, recall_score, roc_auc_score,\
                            plot_confusion_matrix, classification_report, plot_roc_curve, f1_score

import plotly 
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import iplot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import warnings
warnings.filterwarnings("ignore")
print("Successful")

Successful


# Upload dataset

In [50]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/YiyiWang7788/HHA550_Diabetes_Prediction-/main/Dataset/Clinical%20Diabetes%20Data%20Final%20Project%20(1).csv?token=GHSAT0AAAAAABS3LUOHBCNX4RWLUYIA5WUCYSUVNKQ')
diabetes

,race,gender,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diag0ses,metformin,insulin,change,readmitted,readmit_yn
0,2,1,5,1,41,0,1,1,0,0,0,0,0
1,2,1,15,3,59,0,18,9,0,1,1,2,1
2,1,1,25,2,11,5,13,6,0,0,0,0,0
3,2,2,35,2,44,1,16,7,0,1,1,0,0
4,2,2,45,1,51,0,8,5,0,3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,1,2,75,3,51,0,16,9,3,2,1,2,1
101762,1,1,85,5,33,3,18,9,0,3,0,0,0
101763,2,2,75,1,53,0,9,13,3,2,1,0,0
101764,2,1,85,10,45,2,21,9,0,1,1,0,0


# Breaking the data up into Train & Test

In [51]:
train_df, valid_df, test_df = np.split(diabetes.sample(frac=1, random_state=42), 
                                       [int(.7*len(diabetes)), int(0.85*len(diabetes))])
train_df = train_df.reset_index(drop = True)
valid_df = valid_df.reset_index(drop = True)
test_df = test_df.reset_index(drop = True)

In [52]:
diabetes.readmit_yn.value_counts()
# 54864 patients not readimitted to hospitals, 46902 patients had readmitted to hospitals. The number is close to balance. 

0    54864
1    46902
Name: readmit_yn, dtype: int64

In [53]:
diabetes.readmitted.value_counts()

0    54864
2    35545
1    11357
Name: readmitted, dtype: int64

In [54]:
train_df.readmit_yn.value_counts()

0    38357
1    32879
Name: readmit_yn, dtype: int64

In [55]:
valid_df.readmit_yn.value_counts()

0    8263
1    7002
Name: readmit_yn, dtype: int64

In [56]:
test_df.readmit_yn.value_counts()

0    8244
1    7021
Name: readmit_yn, dtype: int64

# Treating the Imbalance in the Data

In [57]:
def calc_prevalence(y_actual):
    
    '''
    This function is to understand the ratio/distribution of the classes that we are going to predict for.
    
    Params:
    1. y_actual: The target feature
    
    Return:
    1. (sum(y_actual)/len(y_actual)): The ratio of the postive class in the comlpete data.
    '''
    
    return (sum(y_actual)/len(y_actual))

In [58]:
# split the training data into positive and negative
rows_pos = train_df.readmit_yn == 1
df_train_pos = train_df.loc[rows_pos]
df_train_neg = train_df.loc[~rows_pos]

# merge the balanced data
readmit_df_balanced = pd.concat([df_train_pos, df_train_neg.sample(n = len(df_train_pos), random_state = 111)],axis = 0)

# shuffle the order of training samples 
readmit_df_balanced = readmit_df_balanced.sample(n = len(readmit_df_balanced), random_state = 42).reset_index(drop = True)

print('Train balanced prevalence(n = %d):%.3f'%(len(readmit_df_balanced), \
                                                calc_prevalence(readmit_df_balanced.readmit_yn.values)))

Train balanced prevalence(n = 65758):0.500


In [59]:
readmit_df_balanced.readmit_yn.value_counts()

0    32879
1    32879
Name: readmit_yn, dtype: int64

In [60]:
X_train = readmit_df_balanced.drop('readmit_yn',axis=1)

y_train = readmit_df_balanced['readmit_yn']

X_valid = valid_df.drop('readmit_yn',axis=1)

y_valid = valid_df['readmit_yn']

X_test = test_df.drop('readmit_yn',axis=1)

y_test = test_df['readmit_yn']

In [61]:
diabetes.columns

Index(['race', 'gender', 'age', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_diag0ses', 'metformin',
       'insulin', 'change', 'readmitted', 'readmit_yn'],
      dtype='object')

In [62]:
scaler=StandardScaler()
X_train[['gender', 'age', 'number_diag0ses']] = pd.DataFrame(scaler.fit_transform(X_train[['gender', 'age', 'number_diag0ses']]),columns=['gender', 'age', 'number_diag0ses'])
X_valid[['gender', 'age', 'number_diag0ses']] = pd.DataFrame(scaler.transform(X_valid[['gender', 'age', 'number_diag0ses']]),columns=['gender', 'age', 'number_diag0ses'])
X_test[['gender', 'age', 'number_diag0ses']] = pd.DataFrame(scaler.transform(X_test[['gender', 'age', 'number_diag0ses']]),columns=['gender', 'age', 'number_diag0ses'])

# Creating and Understanding Models

In [63]:
def calc_specificity(y_actual, y_pred, thresh):
    # calculates specificity
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

def print_report(y_actual, y_pred, thresh = 0.5):
    
    '''
    This function calculates all the metrics to asses the machine learning models.
    
    Params:
    1. y_actual: The actual values for the target variable.
    2. y_pred: The predicted values for the target variable.
    3. thresh: The threshold for the probability to be considered as a positive class. Default value 0.5
    
    Return:
    1. AUC
    2. Accuracy
    3. Recall
    4. Precision
    5. Specificity
    '''
    
    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > thresh))
    recall = recall_score(y_actual, (y_pred > thresh))
    precision = precision_score(y_actual, (y_pred > thresh))
    specificity = calc_specificity(y_actual, y_pred, thresh)
    print('AUC:%.3f'%auc)
    print('accuracy:%.3f'%accuracy)
    print('recall:%.3f'%recall)
    print('precision:%.3f'%precision)
    print('specificity:%.3f'%specificity)
    print('prevalence:%.3f'%calc_prevalence(y_actual))
    print(' ')
    return auc, accuracy, recall, precision, specificity

## Linear Regression

In [64]:
lnr = LinearRegression()
lnr.fit(X_train, y_train)

y_valid_preds = lnr.predict(X_valid)
y_valid_preds

array([1.07641049, 1.05734696, 1.09170935, ..., 1.06840169, 1.05299515,
       0.06825697])

#### The linear Regression is positive, based on the outputs array([1.07641049, 1.05734696, 1.09170935, ..., 1.06840169, 1.05299515, 0.06825697]). 

## Logistic Regression

In [37]:
lr=LogisticRegression(random_state = 42, solver = 'newton-cg', max_iter = 200)
lr.fit(X_train, y_train)

y_valid_preds = lr.predict_proba(X_valid)[:,1]

print('Metrics for Validation data:')

lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity = print_report(y_valid,y_valid_preds, 0.5)

Metrics for Validation data:
AUC:1.000
accuracy:1.000
recall:1.000
precision:1.000
specificity:1.000
prevalence:0.459
 


#### the logistic regression shows the test is accurate and precise based on the results of 1 for AUC, accuracy, recall, precision, and specificity. 